In [ ]:
%pip install simpletransformers

In [ ]:
%pip install torch

In [3]:
import pandas as pd
from pandas import Timestamp
import numpy as np
from operator import itemgetter
from datetime import datetime
import re
from scipy import sparse

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


import simpletransformers
from simpletransformers.classification import ClassificationModel
import logging

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
df = pd.read_csv('df2_sent_')

In [5]:
df = df[['subreddit', 'text']]

In [6]:
df.head()

,subreddit,text
0,Anxiety,people who have lived with anxiety long term w...
1,Anxiety,arm body twitches whenever i am having about t...
2,Anxiety,coping techniques i am a freshman in high scho...
3,Anxiety,at what point do you surrender to the anxiety ...
4,Anxiety,shower anxiety - any tips i have been having s...


In [7]:
df.shape

(785639, 2)

In [8]:
df.isnull().sum()

subreddit    0
text         0
dtype: int64

In [9]:
df.dropna(inplace=True) # Drop NaNs.
df.shape

(785639, 2)

In [30]:
df.subreddit.value_counts(normalize=True)

depression      0.526303
SuicideWatch    0.202635
Anxiety         0.179576
bipolar         0.091486
Name: subreddit, dtype: float64

In [31]:
# # Testing the model on a sub-set of the data

# positions = {"depression": 0.526303, "Anxiety": 0.179576, "SuicideWatch": 0.202635, "bipolar": 0.091486}

# total = 10000

# df2 = pd.concat([df[df['subreddit'] == k].sample(int(v * total), replace=False) for k, v in positions.items()])

# print(df2.shape)

# # Put the labels into binary form
# d = dict(zip(df2.subreddit.unique(), range(0,4)))

# df2['label'] = df2.subreddit.map(d, na_action='ignore')

# # Make the train, test and eval sets
# train, test = np.split(df2.sample(frac=1), [int(.8*len(df2))])

# train_df = pd.DataFrame({
#     0: train['text'].replace(r'\n', ' ', regex=True),
#     1: train['label']
# })

# train_df.reset_index(drop=True, inplace=True)
# train_df.head()

# eval_df = pd.DataFrame({
#     0: test['text'].replace(r'\n', ' ', regex=True),
#     1: test['label']
# })

# eval_df.reset_index(drop=True, inplace=True)
# eval_df.head()

In [32]:
# Put the labels into binary form
d = dict(zip(df.subreddit.unique(), range(0, 4)))

df['label'] = df.subreddit.map(d, na_action='ignore')

d

{'Anxiety': 0, 'bipolar': 1, 'depression': 2, 'SuicideWatch': 3}

In [33]:
# Make the train, test and eval sets
train, test = np.split(df.sample(frac=1), [int(.8*len(df))])

train_df = pd.DataFrame({
    0: train['text'].replace(r'\n', ' ', regex=True),
    1: train['label']
})

train_df.reset_index(drop=True, inplace=True)
train_df.head()

,0,1
0,i just do not know anymore please help me plea...,3
1,ever feel like your body is on fire i have not...,1
2,i do not understand why i am alive we are stuc...,2
3,i planned to jump off a building this friday i...,3
4,anyone else get chest pains and shivers horrib...,2


In [34]:
eval_df = pd.DataFrame({
    0: test['text'].replace(r'\n', ' ', regex=True),
    1: test['label']
})

eval_df.reset_index(drop=True, inplace=True)
eval_df.head()

,0,1
0,brother broke down to me after a near attempt ...,3
1,years later its been years since i got diagnos...,2
2,alone i am a failure and a disgrace i would do...,2
3,most people just will not understand what it i...,0
4,how do you explain bipolar disorder to a co-wo...,1


In [35]:
# self.args = {
#    'model_type':  'roberta',
#    'model_name': 'roberta-base',
#    'output_dir': 'outputs/',
#    'cache_dir': 'cache/',
#    'fp16': True,
#    'fp16_opt_level': 'O1',
#    'max_seq_length': 128,
#    'train_batch_size': 8,
#    'eval_batch_size': 8,
#    'gradient_accumulation_steps': 1,
#    'num_train_epochs': 1,
#    'weight_decay': 0,
#    'learning_rate': 4e-5,
#    'adam_epsilon': 1e-8,
#    'warmup_ratio': 0.06,
#    'warmup_steps': 0,
#    'max_grad_norm': 1.0,
#    'logging_steps': 50,
#    'evaluate_during_training': False,
#    'save_steps': 2000,
#    'eval_all_checkpoints': True,
#    'use_tensorboard': True,
#    'overwrite_output_dir': False,
#    'reprocess_input_data': False,
# }

In [36]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [37]:
train_df.shape

(628511, 2)

In [38]:
train_df[1].value_counts(normalize=True)

2    0.526651
3    0.202561
0    0.179378
1    0.091410
Name: 1, dtype: float64

In [39]:
model = ClassificationModel('roberta', 'roberta-base', use_cuda=True, num_labels=4,
                          args={'num_train_epochs': 1, 'overwrite_output_dir': True, 'save_steps': 10000})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [40]:
# Train the model 
model.train_model(train_df, args={'fp16': True})

/home/ubuntu/anaconda3/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:353: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/home/ubuntu/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


(78564, 0.6158876524727994)

In [ ]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [41]:
import sklearn

result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:851: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.6565433196337399, 'acc': 0.7834758922661779, 'eval_loss': 0.5547711789023062}


In [42]:
result

{'mcc': 0.6565433196337399,
 'acc': 0.7834758922661779,
 'eval_loss': 0.5547711789023062}

In [43]:
d

{'Anxiety': 0, 'bipolar': 1, 'depression': 2, 'SuicideWatch': 3}

In [15]:
import simpletransformers
from simpletransformers.classification import ClassificationModel
import logging

In [44]:
'''Import the BERT model'''

model = ClassificationModel('roberta', 'roberta-base', use_cuda=True, num_labels=4,
                          args={'num_train_epochs': 1, 'overwrite_output_dir': True, 'save_steps': 10000})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [45]:
model

In [47]:
test_test = eval_df[0:10000]

In [48]:

result, model_outputs, wrong_predictions = model.eval_model(test_test, acc=sklearn.metrics.accuracy_score)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


KeyboardInterrupt: 

In [42]:
result

{'mcc': 0.6565433196337399,
 'acc': 0.7834758922661779,
 'eval_loss': 0.5547711789023062}